In [10]:
import pandas as pd

In [11]:
"""
red_mercado.py

Esta red neuronal está diseñada para predecir dos eventos en un mercado económico simulado:
1. subida_brusca_precio (subida abrupta de precios)
2. escasez

Cada bloque de código incluye comentarios detallados que explican su propósito y las decisiones de diseño.
"""

'\nred_mercado.py\n\nEsta red neuronal está diseñada para predecir dos eventos en un mercado económico simulado:\n1. subida_brusca_precio (subida abrupta de precios)\n2. escasez\n\nCada bloque de código incluye comentarios detallados que explican su propósito y las decisiones de diseño.\n'

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.utils import class_weight
from keras import Sequential, Input
from keras.layers import Dense, Dropout, Normalization, BatchNormalization
from scikeras.wrappers import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD

In [16]:
# 1. Carga y preparación de datos
# --------------------------------
datos = pd.read_csv("simulacionEconomica.csv")
X = datos.loc[:, 'demanda_preajuste':'volatilidad_precio']
y = datos.loc[:, 'subida_brusca_precio':'escasez']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


In [17]:
# 2. Cálculo de pesos de clases para combatir desbalance
# -----------------------------------------------------
# Calculamos pesos para cada etiqueta por separado (ejemplo para subida_brusca)
weights_subida = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train['subida_brusca_precio']),
    y=y_train['subida_brusca_precio']
)
# Mismodelaría dos etiquetas; aquí aplicaremos manualmente al fit más adelante
class_weights = {0: weights_subida[0], 1: weights_subida[1]}

In [18]:
# 3. Función para construir el modelo (para Grid Search)
# ------------------------------------------------------
def build_model(
    neurons1=64,
    neurons2=32,
    dropout_rate=0.2,
    learning_rate=0.001,
    optimizer='adam'
):
    model = Sequential([
        Input(shape=(5,)),
        Normalization(),
        Dense(neurons1, activation='relu'),
        Dropout(dropout_rate),
        BatchNormalization(),
        Dense(neurons2, activation='relu'),
        Dense(2, activation='sigmoid')
    ])
    # Selección de optimizador y tasa de aprendizaje
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)
    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['binary_accuracy', 'Recall']
    )
    return model



In [44]:
# 4. Envolver con KerasClassifier para usar GridSearchCV
# ------------------------------------------------------
from sklearn.multioutput import MultiOutputClassifier

keras_clf = KerasClassifier(model=build_model, verbose=0)
#multi_clf = MultiOutputClassifier(keras_clf)


In [48]:
# 2) Param grid con prefijos 'estimator__'
param_grid = {
    'model__neurons1':     [32, 64],
    'model__neurons2':     [16, 32],
    'model__dropout_rate': [0.1, 0.2],
    'model__learning_rate':[1e-3, 1e-4],
    'model__optimizer':    ['adam', 'sgd'],
    'batch_size':          [64, 128],
    'epochs':              [10, 20]
}



In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization, Normalization
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, make_scorer

# 1. Carga y partición de datos
datos = pd.read_csv("simulacionEconomica.csv")
X = datos.loc[:, 'demanda_preajuste':'volatilidad_precio']
y = datos.loc[:, 'subida_brusca_precio':'escasez']

X_train, X_test, y_train_df, y_test_df = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Definición del modelo (salida única)
def build_model(neurons1=64,
                neurons2=32,
                dropout_rate=0.2,
                learning_rate=1e-3,
                optimizer='adam'):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Normalization(),
        Dense(neurons1, activation='relu'),
        Dropout(dropout_rate),
        BatchNormalization(),
        Dense(neurons2, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    opt = Adam(learning_rate) if optimizer=='adam' else SGD(learning_rate)
    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['AUC']
    )
    return model

# 3. Parámetros de búsqueda comunes
param_grid_justificada = {
    'model__neurons1': [32, 64],           # Capa oculta principal pequeña y mediana
    'model__dropout_rate': [0.2],     # Dropout bajo y moderado
    'model__learning_rate': [1e-3, 1e-4],  # Learning rates comunes en DL
    'model__optimizer': ['adam'],           # Optimización adaptativa confiable
    'batch_size': [64],                     # Tamaño estándar para batch
    'epochs': [15]                         # Número de épocas razonable para ver tendencia
}

# 4. Función para ejecutar una búsqueda por etiqueta
def run_grid_search(label_name):
    # 4.1 pesos de clase
    w = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train_df[label_name]),
        y=y_train_df[label_name]
    )
    cw = {0: w[0], 1: w[1]}

    # 4.2 definir wrapper
    clf = KerasClassifier(
        model=build_model,
        verbose=0
    )

    # 4.3 GridSearchCV con AUC binario
    auc_scorer = make_scorer(roc_auc_score, needs_proba=True)
    es = EarlyStopping(monitor='val_auc', mode='max', patience=3, restore_best_weights=True)
    grid = GridSearchCV(
        estimator=clf,
        param_grid=param_grid,
        scoring=auc_scorer,
        cv=2,
        verbose=2,
        error_score='raise'
    )

    # 4.4 fit sobre la columna correspondiente
    grid.fit(
        X_train,
        y_train_df[label_name],
        validation_split=0.2,
        callbacks=[es],
        class_weight=cw
    )
    return grid

# 5. Ejecutar ambas búsquedas
grid_subida = run_grid_search('subida_brusca_precio')
grid_escasez = run_grid_search('escasez')

# 6. Extraer cv_results_ y formar DataFrames
df1 = pd.DataFrame(grid_subida.cv_results_)
df2 = pd.DataFrame(grid_escasez.cv_results_)

# 7. Filtrar columnas de interés
params = [c for c in df1.columns if c.startswith('param_')]
df1 = df1[params + ['mean_test_score']].rename(
    columns={'mean_test_score':'score_subida'}
)
df2 = df2[params + ['mean_test_score']].rename(
    columns={'mean_test_score':'score_escasez'}
)

# 8. Merge y cálculo de score combinado
df = pd.merge(df1, df2, on=params)
df['mean_score'] = (df['score_subida'] + df['score_escasez']) / 2

# 9. Seleccionar el mejor
best_row = df.sort_values('mean_score', ascending=False).iloc[0]
best_params = {k.replace('param_', ''): best_row[k] for k in params}
best_mean_score = best_row['mean_score']

print("Mejores hiperparámetros combinados:")
for k, v in best_params.items():
    print(f"  • {k}: {v}")
print(f"Score medio AUC: {best_mean_score:.4f}")


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, model__neurons1=32, model__neurons2=16, model__optimizer=adam; total time=   9.0s
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, model__neurons1=32, model__neurons2=16, model__optimizer=adam; total time=   8.9s
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, model__neurons1=32, model__neurons2=16, model__optimizer=adam; total time=   9.4s
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, model__neurons1=32, model__neurons2=16, model__optimizer=sgd; total time=   7.3s
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, model__neurons1=32, model__neurons2=16, model__optimizer=sgd; total time=   7.9s
[CV] END batch_size=64, epochs=10, model__dropout_rate=0.1, model__learning_rate=0.001, m

In [ ]:
# 7. Resultados de la búsqueda
best_params = grid_result.best_params_
best_score = grid_result.best_score_
print("Mejores hiperparámetros:", best_params)
print(f"Mejor AUC en validación: {best_score:.4f}")

NameError: name 'grid_result' is not defined

In [ ]:
# 8. Evaluación final con el mejor modelo
# ---------------------------------------
best_model = grid_result.best_estimator_.model

y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred_prob, average='macro')

print(f"\nEvaluación en test:")
print(f"Precision global: {accuracy:.4f}")
print(f"Precision macro: {precision:.4f}")
print(f"Recall macro: {recall:.4f}")
print(f"ROC AUC macro: {roc_auc:.4f}")

# Reporte detallado
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['subida_brusca','escasez']))


219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy (macro): 0.7626
Precision (macro): 0.8494
Recall (macro): 0.7718
ROC AUC (macro): 0.9302

Classification Report:
               precision    recall  f1-score   support

subida_brusca       0.77      0.65      0.71      2833
      escasez       0.92      0.89      0.91      1164

    micro avg       0.82      0.72      0.77      3997
    macro avg       0.85      0.77      0.81      3997
 weighted avg       0.82      0.72      0.77      3997
  samples avg       0.26      0.26      0.26      3997

Matriz de confusión — subida brusca:
[[3629  538]
 [ 984 1849]]

Matriz de confusión — escasez:
[[5751   85]
 [ 127 1037]]


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [ ]:
# 9. Predicciones de ejemplo con el mejor modelo
# ---------------------------------------------
muestra = datos.iloc[1510:1520]
prob_muestra = best_model.predict(muestra.loc[:, 'demanda_preajuste':'volatilidad_precio'])
pred_muestra = (prob_muestra >= 0.5).astype(int)

df_pred = pd.DataFrame({
    'demanda_preajuste': muestra.demanda_preajuste,
    'volatilidad_precio': muestra.volatilidad_precio,
    'prob_subida_brusca': prob_muestra[:, 0],
    'prob_escasez': prob_muestra[:, 1],
    'pred_subida_brusca': pred_muestra[:, 0],
    'pred_escasez': pred_muestra[:, 1]
})
print("\nPredicciones de ejemplo:")
print(df_pred.to_string(index=False))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

Predicciones de ejemplo:
 demanda_preajuste  volatilidad_precio  prob_subida_brusca  prob_escasez  pred_subida_brusca  pred_escasez
            2427.0             1.34536            0.726713      0.122132                True         False
            2487.0             1.46544            0.389270      0.015678               False         False
            2519.0             1.53623            0.471926      0.020069               False         False
            2528.0             1.56445            0.285196      0.006426               False         False
            2617.0             1.63936            0.411950      0.013952               False         False
            2688.0             1.82962            0.716722      0.088721                True         False
            2676.0             2.00250            0.722453      0.058455                True         False
            2693.0             2.01184            0.166892      0.001146        